In [31]:
import numpy as np
np.__version__

import math

In [32]:
class TicTacToe:
    def __init__(self):
        self.row_count = 3
        self.col_count = 3
        self.action_size = self.row_count * self.col_count

    def get_initial_state(self):
        ''' Get board with all zeros '''
        return np.zeros((self.row_count, self.col_count))

    def get_next_state(self, state, action, player):
        ''' Get the next state given the given action by the given player '''
        row = action // self.col_count
        col = action % self.col_count
        state[row, col] = player
        return state

    def get_valid_moves(self, state):
        ''' Get all the legal moves in the position '''
        return (state.reshape(-1) == 0).astype(np.uint8)

    def check_win(self, state, action):
        ''' Check if the given action has led to a win '''
        if action == None:
            return False
        
        row = action // self.col_count
        col = action % self.col_count
        player = state[row, col]

        return (
            np.sum(state[row, :]) == player * self.col_count or # rows
            np.sum(state[:, col]) == player * self.row_count or # columns
            np.sum(np.diag(state)) == player * self.row_count or # tl->br diag
            np.sum(np.diag(np.flip(state, axis=0))) == player * self.row_count # tr->bl diag
        )

    def get_value_and_terminated(self, state, action):
        ''' Get the value (win/tie) and if the game has terminated '''
        if self.check_win(state, action):
            return 1, True
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        return 0, False

    def get_opponent(self, player):
        return -player

    def get_opponent_value(self, value):
        return -value

    def change_perspective(self, state, player):
        return state * player

In [37]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken

        self.children = []
        self.expandable_moves = game.get_valid_moves(state)

        self.visit_count = 0
        self.value_sum = 0

    def is_fully_expanded(self):
        return np.sum(self.expandable_moves) == 0 and len(self.children) > 0

    def select(self):
        best_child = None
        best_ucb = -np.inf

        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb

        return best_child

    def get_ucb(self, child):
        q = 1 - ((child.value_sum / child.visit_count) + 1) / 2 # how promising a move is, opp. perspective, normalized on [0,1]
        return q + self.args['C'] * math.sqrt(math.log(self.visit_count) / child.visit_count)

    def expand(self):
        action = np.random.choice(np.where(self.expandable_moves == 1)[0]) # get random, legal move
        self.expandable_moves[action] = 0

        child_state = self.state.copy()
        child_state = self.game.get_next_state(child_state, action, 1)
        child_state = self.game.change_perspective(child_state, player=-1)

        child = Node(self.game, self.args, child_state, self, action)
        self.children.append(child)
        return child

    def simulate(self):
        value, is_terminal = self.game.get_value_and_terminated(self.state, self.action_taken)
        value = self.game.get_opponent_value(value)

        if is_terminal:
            return value

        rollout_state = self.state.copy()
        rollout_player = 1
        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = np.random.choice(np.where(valid_moves == 1)[0])
            rollout_state = self.game.get_next_state(rollout_state, action, rollout_player)
            value, is_terminal = self.game.get_value_and_terminated(rollout_state, action)

            if is_terminal:
                if rollout_player == -1:
                    value = self.game.get_opponent_value(value)
                return value

            rollout_player = self.game.get_opponent(rollout_player)

    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1

        # flip value for opponent (parent)
        value = self.game.get_opponent_value(value)

        if self.parent is not None:
            self.parent.backpropagate(value)

class MCTS:
    def __init__(self, game, args):
        self.game = game
        self.args = args

    def search(self, state):
        # DEFINE ROOT
        root = Node(self.game, self.args, state)
        
        for search in range(self.args['num_searches']):
            node = root
            
            # SELECTION
            while node.is_fully_expanded():
                node = node.select()
                
            # check for end of game
            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            value = self.game.get_opponent_value(value) # flip parent value

            if not is_terminal:
                # EXPANSION
                node = node.expand()
    
                # SIMULATION
                value = node.simulate()
    
            # BACKPROP
            node.backpropagate(value)

        # probabilities of action being good
        action_probs = np.zeros(self.game.action_size)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count

        action_probs /= np.sum(action_probs)
        return action_probs

In [38]:
ttt = TicTacToe()
player = 1

args = {
    'C': math.sqrt(2),
    'num_searches': 1000
}

mcts = MCTS(ttt, args)

state = ttt.get_initial_state()

while True:
    print(state)

    if player == 1:
        valid_moves = ttt.get_valid_moves(state)
        print('Valid Moves:' , [i for i in range(ttt.action_size) if valid_moves[i] == 1])
        action = int(input(f'{player}: '))
    
        if valid_moves[action] == 0:
            print('Invalid move')
            continue
    else:
        neutral_state = ttt.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        action = np.argmax(mcts_probs)

    state = ttt.get_next_state(state, action, player)

    value, is_terminal = ttt.get_value_and_terminated(state, action)

    if is_terminal:
        print(state)
        if value == 1:
            print(player, 'won')
        else:
            print('Draw')
        break

    player = ttt.get_opponent(player)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Valid Moves: [0, 1, 2, 3, 4, 5, 6, 7, 8]


1:  4


[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
[[ 0.  0.  0.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]
Valid Moves: [0, 1, 2, 3, 5, 7, 8]


1:  6


Invalid move
[[ 0.  0.  0.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]
Valid Moves: [0, 1, 2, 3, 5, 7, 8]


...

[[ 0.  0.  0.]
 [ 0.  1.  0.]
 [-1.  1.  0.]]
[[ 0. -1.  0.]
 [ 0.  1.  0.]
 [-1.  1.  0.]]
Valid Moves: [0, 2, 3, 5, 8]


1:  8


[[ 0. -1.  0.]
 [ 0.  1.  0.]
 [-1.  1.  1.]]
[[-1. -1.  0.]
 [ 0.  1.  0.]
 [-1.  1.  1.]]
Valid Moves: [2, 3, 5]


1:  3


[[-1. -1.  0.]
 [ 1.  1.  0.]
 [-1.  1.  1.]]
[[-1. -1. -1.]
 [ 1.  1.  0.]
 [-1.  1.  1.]]
-1 won
